In [1]:

import time
import pandas as pd
import glob
import numpy as np
import timeit
import pickle

from multiprocessing import Pool
from matplotlib import pyplot as plt
def toc(start_time):
    elapsed = timeit.default_timer() - start_time
    print(elapsed)
start_time = timeit.default_timer()

# import matplotlib.pyplot as plt

%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
plt.rcParams.update({'font.size': 8, 'font.style': 'normal', 'font.family':'serif'})
%config InlineBackend.figure_format = 'svg'
plt.rcParams.update({'font.size': 8, 'font.style': 'normal', 'font.family':'serif'})
#matplotlib.rcParams.update({'font.size': 18})



def importPGNData(filepath):
    """This function returns the data read as a string"""
    with open(filepath) as f:
        return f.readlines()
def getEdgePoints(data):
    """This function returns the start and end indices for each game in the PGN"""
    ends=[]
    starts=[]
    for n,l in enumerate(data):
        if l.startswith("[Event"):
            if n!=0:
                ends.append(n - 1)
            starts.append(n)
        elif (n==len(data)-1):
            ends.append(n)

    return (starts,ends)

path_dicts=  # create a path for to save the dictionaries

## Plan--
1. save each game in a dict by 'player_id' and write to pickle file
    - read player name from the game
    - 0th line is event type, 1st is site, 2nd is white player id, 3rd is black player id
result_2. iterate over all months, years
3. merge dicts by 'player_id'

In [2]:
# 'myVar' in locals():
# del event_type,time_control,white_player,black_player,result_,utcdate,utctime,whiteelo,blackelo,opening_,actual_game_;


In [1]:

#----------------------------------------------------
# importing the pgn
#----------------------------------------------------

t_ic=time.time();

YEAR_OF_PGN=2013
for YEAR_OF_PGN in range(2013,2017):
    for MONTH_OF_PGN in range(1,13):
        if MONTH_OF_PGN<10: MONTH_OF_PGN='0'+str(MONTH_OF_PGN)
        # files="/mnt/sdb1/sandeep/chess_data/lichess_db_standard_rated_2018-12.pgn"
        files="lichess_db_standard_rated_"+str(YEAR_OF_PGN)+"-"+str(MONTH_OF_PGN)+".pgn"

        data = importPGNData(files)
        t_oc=time.time();
        print(t_oc-t_ic,' seconds')

        #----------------------------------------------------
        # Getting the begining and ending point of games in the data file
        #----------------------------------------------------

        starts, ends = getEdgePoints(data)
        t_oc=time.time();
        print(t_oc-t_ic,' seconds')

        #----------------------------------------------------
        #Making dictionary {player name : [game details]}
        #----------------------------------------------------
        dict_career={}
        #31179146
        LL=len(starts)
        for it_game in range(LL):
            if it_game % 50000 == 0:
                t_oc = time.time();
                print('MONTH=',MONTH_OF_PGN,'-----',it_game/LL)

            game=data[starts[it_game]:ends[it_game]]
            flag_actual=0;
            for it in range(len(game)):
                l=game[it]
                if l.startswith("[Event"):            event_type=l.split('"')[1]
                if l.startswith("[White "):            white_player=l.split('"')[1]
                if l.startswith("[Black "):            black_player=l.split('"')[1]            
                if l.startswith("[Result"):            result_=l.split('"')[1]
                if l.startswith("[WhiteElo"):            whiteelo=l.split('"')[1]
                if l.startswith("[BlackElo"):            blackelo=l.split('"')[1]
                if l.startswith("[UTCDate"):            utcdate=l.split('"')[1]
                if l.startswith("[UTCTime"):            utctime=l.split('"')[1]
                if l.startswith("[WhiteRatingDiff"):            wratdiff=l.split('"')[1]
                if l.startswith("[BlackRatingDiff"):            bratdiff=l.split('"')[1]
                if l.startswith("[ECO"):            eco_=l.split('"')[1]
                if l.startswith("[Opening"):            opening_=l.split('"')[1]
                if l.startswith("[TimeControl"):            time_control=l.split('"')[1]            
                if l.startswith("[Termination"):            termination_=l.split('"')[1]            

                if l.startswith("1."):            actual_game_=l;flag_actual=1; break

            if flag_actual==1:

                everything_about_game=[event_type,time_control,white_player,black_player,result_,utcdate,utctime,whiteelo,blackelo,opening_,actual_game_,termination_]    


                if white_player in dict_career.keys():
                    dict_career[white_player].append(everything_about_game)
                else:
    #                 dict_career[white_player]=(everything_about_game)
                    dict_career.setdefault(white_player, []).append(everything_about_game)

                if black_player in dict_career.keys():
                    dict_career[black_player].append(everything_about_game)
                else:
    #                 dict_career[black_player]=(everything_about_game)
                    dict_career.setdefault(black_player, []).append(everything_about_game)
                del event_type,time_control,white_player,black_player,result_,utcdate,utctime,whiteelo,blackelo,opening_,actual_game_,termination_;

        #----------------------------------------------------
        # Writing dictionary to file 
        #----------------------------------------------------        
        file_write=path_dicts+"dict_lichess_db_standard_rated_"+str(YEAR_OF_PGN)+"-"+str(MONTH_OF_PGN)+".pkl"

        with open(file_write, 'wb') as f:
            pickle.dump(dict_career, f, pickle.HIGHEST_PROTOCOL)

        t_oc=time.time();
        print(t_oc-t_ic,' seconds')